In [1]:
# tell jupyter where pyspark is
import findspark
findspark.init()
print('findspark initialized-server')

findspark initialized-server


In [6]:
#Setup Twitter API
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

import socket
import sys
import requests
import requests_oauthlib
import json

#Add variables used for OAuth for connecting to Twitter
ACCESS_TOKEN='45462402-8w8hxSzhVklgOeBoazZS2pVWi8voUUuzK9AImEwYM'
ACCESS_SECRET='lUmy1dUqVwuoDb5OA8GpEPf7hkrDeTMQEZqpd7CGcO7vw'
CONSUMER_KEY='Lnb7WdeLynNONB1kYwYZbD27t'
CONSUMER_SECRET='Y05qZB1JdEtYrstEKgt6sqmUokiujEVTA0rANbFfW2cY59B4FW'

class listener(StreamListener):
    def on_data(self, data):
        print(data)
        return(True)
    def on_error(self, status):
        print(status)
    
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
twitterStream = Stream(auth, listener())
twitterStream.filter(track=["#blibli", "#tokopedia", "#lazada", "#jdid", "#bukalapak", "#shopee", "#redmi", "#redmi6a","#redmiy2","#redmi6pro"])
#twitterStream.filter(track=["hive","spark"])
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)

KeyboardInterrupt: 

+----------+-------------+
|   hashtag|hashtag_count|
+----------+-------------+
|#CareerArc|         7906|
|      #job|         6714|
|   #Hiring|         6148|
|  #hiring!|         2228|
|      #Job|         2097|
|     #Jobs|         2096|
|   #hiring|         1284|
|     #job:|         1101|
|     #job?|         1082|
|  #Dodgers|          558|
+----------+-------------+



In [2]:
###Function definitions
#Process totals in RDD
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
        send_df_to_dashboard(hashtag_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

#Send the hashtag_counts_df data frame to the dashboard
def send_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
    # initialize and send the data through REST API
    url = 'http://big-data-analytics:5000/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [3]:
#Spark
from pyspark import SparkConf,SparkContext
#Spark Streaming
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext

import sys
import requests

# create spark configuration
conf = SparkConf()
conf.setMaster('local[2]')
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
#sc.setLogLevel("WARN")

#Create local streamingContext with two working thread and batch interval of 1 second
#Passing Spark Context into Streaming, along with batch duration (i.e. 60 seconds)
ssc = StreamingContext(sc,300)

# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")
# read data from port 9009
dataStream = ssc.socketTextStream("big-data-analytics",9009)

In [5]:
###Twitter streaming corpus transformation

# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling None.org.apache.spark.streaming.api.python.PythonTransformedDStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:224)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:66)
	at org.apache.spark.streaming.api.python.PythonDStream.<init>(PythonDStream.scala:224)
	at org.apache.spark.streaming.api.python.PythonTransformedDStream.<init>(PythonDStream.scala:241)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
